In [11]:
import time 
import requests
from time import sleep
import re,requests,csv
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium import webdriver
keys = Keys()
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [12]:
browser = webdriver.Chrome()

In [13]:
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',"accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7" }

## 검색어 입력

In [14]:
keyword = input("검색하시고자 하는 상품을 입력하세요.:")
url = 'https://www.coupang.com/np/search?q={}'
final_url = url.format(keyword)
browser.get(final_url)

검색하시고자 하는 상품을 입력하세요.: 강아지 사료


In [24]:
links = browser.find_elements(By.XPATH,"//*[@id='productList']")
links

[<selenium.webdriver.remote.webelement.WebElement (session="497ec82d70198534353e6539bb1f18a9", element="C2045903BF9A3A3949C2A6F2DB2295FD_element_151")>]

In [15]:
product_list = browser.find_element(By.XPATH, "//ul[@id='productList']")
product_list

<selenium.webdriver.remote.webelement.WebElement (session="4aedf14722628f7a956912ff4081b7b6", element="3FC8241B014C9F6B29ECF02D381772E6_element_141")>

In [16]:
soup = BeautifulSoup(product_list.get_attribute('outerHTML'),'html.parser')
soup

<ul class="search-product-list" data-is-loyalty-member="false" data-products='{"productSizePerPage":21, "bundleId":"62", "indexes":[7387472273, 1285967491, 62318395, 1285967491, 7387472273, 1550850914, 392378, 7867509931, 7824063246, 1285967491, 7481849014, 7594500365, 7434900810, 6954553920, 392378, 7655727336, 6954553920, 5998144736, 6080929259, 115784832, 7303642614, 7346975754, 7170134497, 7592432810, 5625145118, 822882, 7573803732, 6079713850, 5649856239, 62318395, 7486032733, 1550850914, 78748094, 1285967491, 392378, 5998144736] }' data-query="강아지 사료" data-search-id="2795332740d64eeb831cda5d8532c3d0" id="productList">
<li class="search-product search-product__ad-badge" data-coupon-nudge-text="" data-coupon-tag-area="true" data-freebie-vendor-item-id="null" data-handyman-area="" data-is-rocket="true" data-is-soldout="" data-product-id="7387472273" data-vendor-item-id="84932792521" data-winner-vendor-item-id="84932792521" id="7387472273"><a class="search-product-link" data-is-soldo

## 검색어에 대한 제품 URL_LIST 반환

In [17]:
url_list = []

for product in soup.find_all('li', class_='search-product'):
    # 광고 상품인 경우 스킵
    if 'search-product__ad-badge' in product['class']:
        continue
    product_url = "https://www.coupang.com" + product.find('a', class_='search-product-link')['href']
    url_list.append(product_url)

print(url_list)
len(url_list)

['https://www.coupang.com/vp/products/1285967491?itemId=18378213622&vendorItemId=86182850368&pickType=COU_PICK', 'https://www.coupang.com/vp/products/62318395?itemId=212802869&vendorItemId=70248727064', 'https://www.coupang.com/vp/products/1285967491?itemId=18378213620&vendorItemId=86182850401', 'https://www.coupang.com/vp/products/392378?itemId=7743177036&vendorItemId=85983928104', 'https://www.coupang.com/vp/products/7867509931?itemId=21483953491&vendorItemId=70372743043', 'https://www.coupang.com/vp/products/1285967491?itemId=19059899051&vendorItemId=86182850370', 'https://www.coupang.com/vp/products/2050342335?itemId=3485750527&vendorItemId=73511550610&sourceType=SDW_TOP_SELLING_WIDGET_V2&searchId=2795332740d64eeb831cda5d8532c3d0&q=강아지 사료', 'https://www.coupang.com/vp/products/1349229727?itemId=2377178366&vendorItemId=70372743051&sourceType=SDW_TOP_SELLING_WIDGET_V2&searchId=2795332740d64eeb831cda5d8532c3d0&q=강아지 사료', 'https://www.coupang.com/vp/products/109613?itemId=209055019&ven

38

## 제품 url 별 상품평 추출 + 데이터프레임으로 저장

In [ ]:
browser = webdriver.Chrome()
review_data = pd.DataFrame()
url = url_list[:2]
for url in url_list[12:]:
    
    browser.get(url)
    browser.page_source
    time.sleep(2)

    # 상품평 버튼 클릭
    button_review = browser.find_elements(By.XPATH,'//*[@id="btfTab"]/ul[1]/li[2]')
    button_review[0].click()
    time.sleep(3)
    
    soup = BeautifulSoup(browser.page_source,'html.parser',from_encoding = 'utf-8')
    time.sleep(6)
    
    # 평점 추출
    star_rate = soup.find('div',class_= 'sdp-review__average__total-star__info-orange js_reviewAverageTotalStarRating')
    total_star_rate = star_rate['data-rating']


    # 전체 리뷰수 추출
    total_review = soup.find('div',class_='sdp-review__average__total-star__info-count')
    total_review = total_review.text
    total_review = total_review.replace(',', '') 
    print(total_review)

    if int(total_review) > 1000:
        click_next_page_num = 20
    else:
        click_next_page_num = int(total_review)//50


    
    data_list = []
    for i in range(0,click_next_page_num): # 페이지 넘기기 횟수 만큼
        button_num = ['button[2]', 'button[3]','button[4]','button[5]','button[6]','button[7]','button[8]','button[9]','button[10]','button[11]',
              'button[12]']
        for page_num in button_num: #한 페이지에 있는 버튼 개수만큼 
            print(page_num)

            button_page = browser.find_elements(By.XPATH,'//*[@id="btfTab"]/ul[2]/li[2]/div/div[6]/section[4]/div[3]/'+ page_num)
            print(button_page)
            time.sleep(2)
            button_page[0].click() 
            wait = WebDriverWait(browser, 15)
            wait.until(EC.staleness_of(button_page[0]))

            soup = BeautifulSoup(browser.page_source,'html.parser',from_encoding = 'utf-8')

            #페이지 내 리뷰 추출
            articles = soup.findAll('article',class_='sdp-review__article__list js_reviewArticleReviewList')
            
            for a in articles:
                productnametag=a.find('div', class_='sdp-review__article__list__info__product-info__name') 
                total_star_rate = total_star_rate
                headlinetag=a.find('div', class_='sdp-review__article__list__headline') 
                contenttag=a.find('div', class_='sdp-review__article__list__review__content js_reviewArticleContent')
                help = a.find('div',class_ = 'sdp-review__article__list__help js_reviewArticleHelpfulContainer')
                help_count = help['data-count']
                star_rate = a.find('div',class_ = 'sdp-review__article__list__info__product-info__star-orange js_reviewArticleRatingValue')
                star_rate = star_rate['data-rating']
                
                productname=''
                headline=''
                content=''

                # 오류 방지
                if(productnametag!=None):
                    product_name=productnametag.text
                if(headlinetag!=None):
                    headline=headlinetag.text
                if(contenttag!=None):
                    content=contenttag.text
                if(help_count!=None):
                    help_count=help_count
                if(total_star_rate!=None):
                    total_star_rate = total_star_rate
                if(star_rate!=None):
                    star_rate = star_rate
                data = {"product_name" : product_name, "total_star_rate": total_star_rate, "title" : headline, "content" : content,"help_count": help_count,'star_rate': star_rate}
                data_list.append(data)
                print('-----------------------------------------',product_name)
                print(headline,content,help_count,total_star_rate,star_rate)
    df = pd.DataFrame(data_list)
    review_data = pd.concat([review_data,df],ignore_index = True)
            
print(review_data)

In [ ]:
result = pd.DataFrame(review_data)
result

In [ ]:
review_data.to_csv('review_dog.csv',encoding = 'utf-8')